In [1]:
import cv2
import numpy as np
import os
import math
from scipy.spatial import distance as dist
from collections import OrderedDict
from scipy.optimize import linear_sum_assignment
from kalman_utils.KFilter import *

In [ ]:
from filterpy.kalman import KalmanFilter, UnscentedKalmanFilter, MerweScaledSigmaPoints
from filterpy.common import Q_discrete_white_noise

In [22]:
a = [[170, 175, 196, 209], [150, 557, 174, 577], [625, 194, 640, 209], [170, 175, 196, 209], [173, 225, 202, 253], [435, 526, 476, 568], [435, 576, 476, 603]]

b = np.array([[170, 175, 196, 209], [150, 557, 174, 577], [625, 194, 640, 209], [170, 175, 196, 209], [173, 225, 202, 253], [435, 526, 476, 568], [435, 576, 476, 603]])

no_ball_box = list(set([tuple(set(i)) for i in a]))

In [15]:
no_ball_box

[(640, 625, 194, 209),
 (577, 174, 557, 150),
 (225, 202, 173, 253),
 (568, 435, 476, 526),
 (209, 170, 196, 175),
 (576, 603, 435, 476)]

In [16]:
for i in a:
    print(set(i))

[170, 175, 196, 209]
[150, 557, 174, 577]
[625, 194, 640, 209]
[170, 175, 196, 209]
[173, 225, 202, 253]
[435, 526, 476, 568]
[435, 576, 476, 603]


In [17]:
for i in a:
    print(set(i))

{209, 170, 196, 175}
{577, 174, 557, 150}
{640, 625, 194, 209}
{209, 170, 196, 175}
{225, 202, 173, 253}
{568, 435, 476, 526}
{576, 603, 435, 476}


In [32]:
new_array = [tuple(row) for row in b]

In [35]:
c = np.unique(new_array)

In [36]:
c

array([150, 170, 173, 174, 175, 194, 196, 202, 209, 225, 253, 435, 476,
       526, 557, 568, 576, 577, 603, 625, 640])

In [33]:
new_array

[(170, 175, 196, 209),
 (150, 557, 174, 577),
 (625, 194, 640, 209),
 (170, 175, 196, 209),
 (173, 225, 202, 253),
 (435, 526, 476, 568),
 (435, 576, 476, 603)]

In [2]:
['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush'].index("tennis racket")

38

In [4]:
for i in range(640, 1080):
    a = i

    
    if ((9 * a)/16).is_integer():
        print("x : ",i)
        print("y : ",(9 * a)/16)
        print("total_pixel : ", a * (9 * a)/16 *2 )
        

x :  640
y :  360.0
total_pixel :  460800.0
x :  656
y :  369.0
total_pixel :  484128.0
x :  672
y :  378.0
total_pixel :  508032.0
x :  688
y :  387.0
total_pixel :  532512.0
x :  704
y :  396.0
total_pixel :  557568.0
x :  720
y :  405.0
total_pixel :  583200.0
x :  736
y :  414.0
total_pixel :  609408.0
x :  752
y :  423.0
total_pixel :  636192.0
x :  768
y :  432.0
total_pixel :  663552.0
x :  784
y :  441.0
total_pixel :  691488.0
x :  800
y :  450.0
total_pixel :  720000.0
x :  816
y :  459.0
total_pixel :  749088.0
x :  832
y :  468.0
total_pixel :  778752.0
x :  848
y :  477.0
total_pixel :  808992.0
x :  864
y :  486.0
total_pixel :  839808.0
x :  880
y :  495.0
total_pixel :  871200.0
x :  896
y :  504.0
total_pixel :  903168.0
x :  912
y :  513.0
total_pixel :  935712.0
x :  928
y :  522.0
total_pixel :  968832.0
x :  944
y :  531.0
total_pixel :  1002528.0
x :  960
y :  540.0
total_pixel :  1036800.0
x :  976
y :  549.0
total_pixel :  1071648.0
x :  992
y :  558.0
total_pix

In [50]:
1276,600

1.35

In [53]:
x_meter2pix = 23.77 / x_pix_length
y_meter2pix = 10.97 / y_pix_length

In [52]:
y_pix_length, x_pix_length = 600, 1276

In [57]:
x_meter2pix * 1500
810 * y_meter2pix

14.809500000000002

In [3]:
(1500 - 1276)/3

74.66666666666667

In [7]:
def trans_xy(img_ori, point_list):

    for i in range(len(point_list[1])):
        x_cen, y_cen = point_list[1][i]

        point_list[1][i][1] = y_cen - (img_ori.shape[0] / 2)

    return point_list

In [14]:
img = np.zeros([752, 423 * 2, 3])
 
trans_xy(img,a)

ValueError: too many values to unpack (expected 2)

In [38]:
def cal_ball_position(ball_stats_list):
    
    net_length = 13.11
    post_hegith_avg = 1.125
    
    for i in range(len(ball_stats_list)):
    
        ball_distance_list = ball_stats_list[i][0]
        ball_height_list = ball_stats_list[i][1]

        height = sum(ball_height_list) / 2 - post_hegith_avg

        if sum(ball_distance_list) < 13:
            return [np.nan, np.nan, np.nan]

        ball2net_length_x_L = ball_distance_list[0] * np.sin(theta_L)
        ball_position_y_L = ball_distance_list[0] * np.cos(theta_L)

        ball_plate_angle_L = np.arcsin(height / ball2net_length_x_L)

        ball_position_x_L = ball2net_length_x_L * np.cos(ball_plate_angle_L)

        ball2net_length_x_R = ball_distance_list[1] * np.sin(theta_R)
        ball_position_y_R = ball_distance_list[1] * np.cos(theta_R)

        ball_plate_angle_R = np.arcsin(height / ball2net_length_x_R)

        ball_position_x_R = ball2net_length_x_R * np.cos(ball_plate_angle_R)


        """print("theta_L, theta_R : ", np.rad2deg(self.theta_L), np.rad2deg(self.theta_R))
        print("ball_plate_angle_L, ball_plate_angle_R : ", np.rad2deg(ball_plate_angle_L), np.rad2deg(ball_plate_angle_R))
        print([-ball_position_x_L, ball_position_y_L - 6.4, height + 1])
        print([-ball_position_x_R, 6.4 - ball_position_y_R, height + 1])"""

        if theta_L > theta_R:
            ball_position_y = ball_position_y_L - (net_length / 2)

        else :
            ball_position_y = (net_length / 2) - ball_position_y_R

    return [-ball_position_x_L, ball_position_y, height + post_hegith_avg]


In [39]:
def get_depth_height(L_pos, R_pos):
        
    depth_height = []

    cx = 360
    cy = 204
    focal_length = 320.754

    net_length = 13.11

    post_hegith_left = 1.13
    post_hegith_right = 1.12 

    for i in range(len(L_pos)):
        x_L, y_L = L_pos[i][0] - cx, L_pos[i][1] - cy

        for j in range(len(R_pos)):
            x_R, y_R = R_pos[j][0] - cx, R_pos[j][1] - cy


            c_L = np.sqrt(focal_length ** 2 + x_L ** 2 + y_L ** 2)
            a_L = np.sqrt(focal_length ** 2 + x_L ** 2)

            if x_L < 0:
                th_L = 0.785398 + np.arccos(focal_length / a_L)

            else :
                th_L = 0.785398 - np.arccos(focal_length / a_L)


            b_L = a_L * np.cos(th_L)

            c_R = np.sqrt(focal_length ** 2 + x_R ** 2 + y_R ** 2)
            a_R = np.sqrt(focal_length ** 2 + x_R ** 2)

            if x_R > 0:
                th_R = 0.785398 + np.arccos(focal_length / a_R)

            else :
                th_R = 0.785398 - np.arccos(focal_length / a_R)

            b_R = a_R * np.cos(th_R)

            theta_L = np.arccos(b_L/c_L)
            theta_R = np.arccos(b_R/c_R)


            D_L = net_length * np.sin(theta_R) / np.sin(3.14 - (theta_L + theta_R))
            D_R = net_length * np.sin(theta_L) / np.sin(3.14 - (theta_L + theta_R))

            height_L = abs(D_L * np.sin(np.arcsin(y_L/c_L)))
            height_R = abs(D_R * np.sin(np.arcsin(y_R/c_R)))

            #height_L = abs(D_L * np.sin(np.arctan(y_L/a_L)))
            #height_R = abs(D_R * np.sin(np.arctan(y_R/a_R)))

            if y_L < 0:
                height_L += post_hegith_left

            else:
                height_L -= post_hegith_left  


            if y_R < 0:
                height_R += post_hegith_right

            else:
                height_R -= post_hegith_right  


            print(L_pos[i],R_pos[j])
            print([D_L, D_R, height_L, height_R])
            depth_height.append([[D_L, D_R], [height_L, height_R]])

    return depth_height

In [40]:
ball_cen_left =  [[260, 162]]
ball_cen_right =  [[351, 167]]

ball_stats_list = get_depth_height(ball_cen_left,ball_cen_right)

[260, 162] [351, 167]
[9.458568396152303, 12.128951367682397, 2.3032568761005505, 2.509357032389613]


In [41]:
cal_ball_position(ball_stats_list)

NameError: name 'self' is not defined

In [42]:
def get_ball_pos(L_pos, R_pos):
        
    depth_height = []

    cx = 360
    cy = 204
    focal_length = 320.754

    net_length = 13.11

    post_hegith_left = 1.13
    post_hegith_right = 1.12 

    post_hegith_avg = (post_hegith_left + post_hegith_right) / 2

    for i in range(len(L_pos)):
        x_L, y_L = L_pos[i][0] - cx, L_pos[i][1] - cy

        for j in range(len(R_pos)):
            x_R, y_R = R_pos[j][0] - cx, R_pos[j][1] - cy


            c_L = np.sqrt(focal_length ** 2 + x_L ** 2 + y_L ** 2)
            a_L = np.sqrt(focal_length ** 2 + x_L ** 2)

            if x_L < 0:
                th_L = 0.785398 + np.arccos(focal_length / a_L)

            else :
                th_L = 0.785398 - np.arccos(focal_length / a_L)


            b_L = a_L * np.cos(th_L)

            c_R = np.sqrt(focal_length ** 2 + x_R ** 2 + y_R ** 2)
            a_R = np.sqrt(focal_length ** 2 + x_R ** 2)

            if x_R > 0:
                th_R = 0.785398 + np.arccos(focal_length / a_R)

            else :
                th_R = 0.785398 - np.arccos(focal_length / a_R)

            b_R = a_R * np.cos(th_R)

            theta_L = np.arccos(b_L/c_L)
            theta_R = np.arccos(b_R/c_R)


            D_L = net_length * np.sin(theta_R) / np.sin(3.14 - (theta_L + theta_R))
            D_R = net_length * np.sin(theta_L) / np.sin(3.14 - (theta_L + theta_R))

            height_L = abs(D_L * np.sin(np.arcsin(y_L/c_L)))
            height_R = abs(D_R * np.sin(np.arcsin(y_R/c_R)))

            #height_L = abs(D_L * np.sin(np.arctan(y_L/a_L)))
            #height_R = abs(D_R * np.sin(np.arctan(y_R/a_R)))

            if y_L < 0:
                height_L += post_hegith_left

            else:
                height_L -= post_hegith_left  


            if y_R < 0:
                height_R += post_hegith_right

            else:
                height_R -= post_hegith_right  

            ball_height_list = [height_L, height_R]
            ball_distance_list = [D_L, D_R]

            height = sum(ball_height_list) / 2 - post_hegith_avg

            ball2net_length_x_L = ball_distance_list[0] * np.sin(theta_L)
            ball_position_y_L = ball_distance_list[0] * np.cos(theta_L)

            ball_plate_angle_L = np.arcsin(height / ball2net_length_x_L)

            ball_position_x_L = ball2net_length_x_L * np.cos(ball_plate_angle_L)

            ball2net_length_x_R = ball_distance_list[1] * np.sin(theta_R)
            ball_position_y_R = ball_distance_list[1] * np.cos(theta_R)

            ball_plate_angle_R = np.arcsin(height / ball2net_length_x_R)

            ball_position_x_R = ball2net_length_x_R * np.cos(ball_plate_angle_R)

            if theta_L > theta_R:
                ball_position_y = ball_position_y_L - (net_length / 2)

            else :
                ball_position_y = (net_length / 2) - ball_position_y_R


            print(L_pos[i],R_pos[j])
            #print([D_L, D_R, height_L, height_R])
            print([-ball_position_x_L, ball_position_y, height + post_hegith_avg])

            depth_height.append([[D_L, D_R], [height_L, height_R]])

            return [-ball_position_x_L, ball_position_y, height + post_hegith_avg]

In [44]:
ball_cen_left =  [[298, 153]]
ball_cen_right =  [[319, 160]]

ball_stats_list = get_ball_pos(ball_cen_left,ball_cen_right)

[298, 153] [319, 160]
[-6.66652032789926, -2.0337485082041438, 2.493953152765253]


In [ ]:
def check_vel_noise():

    y_vel_list = np.array(esti_ball_val_list)[:,1]


    if len(y_vel_list) > 3 :

        vel_mean = np.mean(y_vel_list)

        if abs(abs(vel_mean) - abs(y_vel_list[-1])) > 2:

            vel_mean = np.mean(y_vel_list[:-1])
            esti_ball_val_list[-1][1] = vel_mean

        return esti_ball_val_list[-1]

    else:
        return esti_ball_val_list[-1]


In [45]:
def cal_landing_point(pos):

    t_list = []

    #vel = self.check_vel_noise()

    x0, y0, z0 = pos[0], pos[1], pos[2]
    vx, vy, vz = vel[0], vel[1], vel[2]

    a = -((0.5 * 0.507 * 1.2041 * np.pi * (0.033 ** 2) * vz ** 2 ) / 0.057 + 9.8 / 2 )
    b = vz
    c = z0

    t_list.append((-b + np.sqrt(b ** 2 - 4 * a * c))/(2 * a))
    t_list.append((-b - np.sqrt(b ** 2 - 4 * a * c))/(2 * a))

    t = max(t_list)

    x = np.array(x0 + vx * t - (0.5 * 0.507 * 1.2041 * np.pi * (0.033 ** 2) * vx ** 2 ) * (t ** 2) / 0.057,float)
    y = np.array(y0 + vy * t - (0.5 * 0.507 * 1.2041 * np.pi * (0.033 ** 2) * vy ** 2 ) * (t ** 2) / 0.057,float)
    z = np.array(z0 + vz * t - ((0.5 * 0.507 * 1.2041 * np.pi * (0.033 ** 2) * vz ** 2 ) / 0.057 + 9.8 / 2) * (t ** 2),float)

    return [np.round(x,3), np.round(y,3), np.round(z,3)]

In [46]:
cal_landing_point(cal_landing_point)

TypeError: 'function' object is not subscriptable

In [49]:
for i in range(3):
    for j in range(4):
        print(j)
        if j == 3:
            break
            
        

0
1
2
3
0
1
2
3
0
1
2
3


In [56]:

class Ball_Pos_Estimation():

    def __init__(self):
        self.pre_ball_cen_left_list = []
        self.pre_ball_cen_right_list = []



    def check_ball_move_update(self, ball_cen_left_list, ball_cen_right_list):

        self.swing_check = True

        left_flag = False
        right_flag = False

        self.ball_cen_left_list = ball_cen_left_list
        self.ball_cen_right_list = ball_cen_right_list

        if len(self.pre_ball_cen_left_list) or len(self.pre_ball_cen_right_list):
            
            for i in range(len(self.ball_cen_left_list)):

                if left_flag:
                    break   

                x_cen = self.ball_cen_left_list[i][0]
                
                for j in range(len(self.pre_ball_cen_left_list)):

                    pre_x_cen = self.pre_ball_cen_left_list[j][0]
                    
                    if x_cen > pre_x_cen:

                        self.pre_ball_cen_left_list = self.ball_cen_left_list
                        left_flag = True
                        break
                
            for i in range(len(self.ball_cen_right_list)):

                if right_flag:
                    break   

                x_cen = self.ball_cen_right_list[i][0]
                
                for j in range(len(self.pre_ball_cen_right_list)):

                    pre_x_cen = self.pre_ball_cen_right_list[j][0]

                    if x_cen < pre_x_cen:

                        self.pre_ball_cen_right_list = self.ball_cen_right_list
                        right_flag = True
                        
                        break
                        
            if left_flag == False and right_flag == False : 
                self.pre_ball_cen_left_list = []
                self.pre_ball_cen_right_list = []
                self.swing_check = False
                return False

            return True

        else:
            self.pre_ball_cen_left_list = self.ball_cen_left_list
            self.pre_ball_cen_right_list = self.ball_cen_right_list

            self.swing_check = False

            return True

In [57]:
estimation_ball = Ball_Pos_Estimation()

In [60]:
ball_cen_left =  [[419, 151]]
ball_cen_right =  [[201, 153]]

if estimation_ball.check_ball_move_update(ball_cen_left, ball_cen_right):
    pass

print(estimation_ball.swing_check)

False


In [61]:
ball_cen_left =  [[392, 160]]
ball_cen_right =  [[223, 160]]

if estimation_ball.check_ball_move_update(ball_cen_left, ball_cen_right):
    pass

print(estimation_ball.swing_check)

False


In [52]:
ball_cen_left =  [[281, 194], [716, 230]]
ball_cen_right =  []

estimation_ball.check_ball_move_update(ball_cen_left, ball_cen_right)

True

In [65]:
ball_pos_list = [np.nan,np.nan,np.nan]

In [67]:
np.isnan(ball_pos_list[0])

True

In [69]:
a = 2

if a == 1:
    print(1)
    
elif a == 2:
    print(2)

2


In [125]:


def fx(x, dt):
    # state transition function - predict next state based
    # on constant velocity model x = vt + x_0

    F = np.matrix([[1.0, 0.0, 0.0, dt, 0.0, 0.0, 1/2.0*dt**2, 0.0, 0.0],
              [0.0, 1.0, 0.0, 0.0,  dt, 0.0, 0.0, 1/2.0*dt**2, 0.0],
              [0.0, 0.0, 1.0, 0.0, 0.0,  dt, 0.0, 0.0, 1/2.0*dt**2],
              [0.0, 0.0, 0.0, 1.0, 0.0, 0.0,  dt, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,  dt, 0.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,  dt],
              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]])

    return np.dot(F,x)

def hx(x):
    # measurement function - convert state into a measurement
    # where measurements are [x_pos, y_pos]

    
    H = np.matrix([[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                   [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                   [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]])
    
    return np.array([x[0], x[3], x[6]])

class UK_filter():

    def __init__(self, dt, std_acc, x_std_meas, y_std_meas,z_std_meas, init_x, init_y, init_z):  
    
        self.init_x = init_x 
        self.init_y = init_y
        self.init_z = init_z
        
        self.dt = dt
        self.z_std = 0.1

        self.points = MerweScaledSigmaPoints(9, alpha=.1, beta=2., kappa=-1)
        self.f = UnscentedKalmanFilter(dim_x=9, dim_z=3, dt=self.dt, fx=fx, hx=hx, points=self.points)

        self.f.x = np.array([self.init_x,0,0,self.init_y,0,0, self.init_z, 0,0])


        self.f.P = np.eye(9)

        self.f.Q = np.matrix([[(dt**6)/36, 0, 0, (dt**5)/12, 0, 0, (dt**4)/6, 0, 0],
                           [0, (dt**6)/36, 0, 0, (dt**5)/12, 0, 0, (dt**4)/6, 0],
                           [0, 0, (dt**6)/36, 0, 0, (dt**5)/12, 0, 0, (dt**4)/6],
                           [(dt**5)/12, 0, 0, (dt**4)/4, 0, 0, (dt**3)/2, 0, 0],
                           [0, (dt**5)/12, 0, 0, (dt**4)/4, 0, 0, (dt**3)/2, 0],
                           [0, 0, (dt**5)/12, 0, 0, (dt**4)/4, 0, 0, (dt**3)/2],
                           [(dt**4)/6, 0, 0, (dt**3)/2, 0, 0, (dt**2), 0, 0],
                           [0, (dt**4)/6, 0, 0, (dt**3)/2, 0, 0, (dt**2), 0],
                           [0, 0, (dt**4)/6, 0, 0, (dt**3)/2, 0, 0, (dt**2)]]) *std_acc**2


        #self.f.Q = Q_discrete_white_noise(2, dt = self.dt, var = 0.01**2, block_size = 2)


        self.f.R = np.array([[x_std_meas**2, 0, 0],
                            [0, y_std_meas**2, 0],
                            [0, 0, z_std_meas**2]])

        #self.f.predict()

In [126]:
a = UK_filter(dt = 0.1, std_acc = 10, x_std_meas = 1, y_std_meas = 1,z_std_meas = 1,
              init_x = 0, init_y = 0, init_z = 0)

In [144]:
a.f.predict()

In [128]:
a.f.update([1,1,1])

In [145]:
a.f.x.reshape([3,3])

array([[ 1.63888382e+01,  8.46852953e-16, -8.95117314e-16],
       [ 2.07578141e+01,  1.57245907e-15,  5.55111512e-16],
       [ 2.66005113e+01,  6.79577922e-16,  5.29683669e-17]])

In [131]:
a.f.update([2,2,2])

In [137]:
a.f.update([10,10,10])

In [140]:
a.f.update([20,20,20])

In [143]:
a.f.update([30,30,30])

In [146]:
ball_cand_trajectory =  [[], [[[-8.499031225886979, 0.311750401425118, 1.404671677765355]]], [[[-7.654383459951477, 0.37339492038427213, 1.528884790477008]]], [[[-6.812550514254101, 0.4358689710714039, 1.6515142171759307]]], [[[-6.005829995239098, 0.5240465335704965, 1.7282085158171043]]], [[[-5.174512338451282, 0.5984737933774342, 1.802634978706663]]], [[[-4.387167345972433, 0.6819632256971389, 1.8761140620728325]]], [[[-3.625711866264724, 0.7480947422039854, 1.9102820296223895]]], [[[-2.8421647085007566, 0.8618655406628255, 1.950691295218776]]], [[[-2.058537458662639, 1.064584863759384, 1.9592570246492662]]], [[]], [[]], [[]]]

In [147]:
len(ball_cand_trajectory)

13

In [156]:
ball_cand_trajectory[3][0][0]

[-6.812550514254101, 0.4358689710714039, 1.6515142171759307]

In [157]:
ball_pos_list = [[-1,0,0], [1,3,4],[2,4,5]]

In [179]:
b = np.array(ball_pos_list)

In [180]:
b[:,0].argmin()

0

In [181]:
b[:][0]

array([-1,  0,  0])

In [173]:
a = [0, 0, 0]
b = [1, 2, 3]



def get_distance(point_1, point_2):

    return (np.sqrt((point_2[0]-point_1[0])**2 + (point_2[1]-point_1[1])**2 + (point_2[2]-point_1[2])**2))


In [175]:
get_distance(a,b)

3.7416573867739413

In [182]:
ball_pos_list = [[-7.923465928004007, -0.6755867599611189, 2.580941671512611]]

In [189]:
x_pos, y_pos, z_pos = ball_pos_list[np.array(ball_pos_list)[:,0].argmin()]


In [188]:
np.array(ball_pos_list)[:,0].argmin()


0

In [190]:
x_pos, y_pos, z_pos

(-7.923465928004007, -0.6755867599611189, 2.580941671512611)

In [4]:
a = np.array([[1,2,3],[0,-5,0],[-8,0,0]])


In [201]:
a.append([[1,2],[3,4]])

In [202]:
a

[[1, 2, 3], [[1, 2], [3, 4]]]

In [1]:
from kalman_utils.KFilter import *

In [2]:
dT = 1 / 25

In [3]:
ball_pos =  [-9.285799665284836, -1.5959832449913565, 2.874695965876609]

In [4]:
kf = Kalman_filiter(ball_pos[0], ball_pos[1], ball_pos[2], dT)

nstates 7
transitionMatrix: shape:(7, 7)
[[1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]]
measurementMatrix: shape:(3, 7)
[[1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]]
processNoiseCov: shape:(7, 7)
[[1.e-06 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
 [0.e+00 1.e-06 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 1.e-06 0.e+00 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 0.e+00 8.e+00 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 0.e+00 0.e+00 8.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 8.e+00 0.e+00]
 [0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 1.e-06]]
measurementNoiseCov: shape:(3, 3)
[[1.e-06 0.e+00 0.e+00]
 [0.e+00 1.e-06 0.e+00]
 [0.e+00 0.e+00 1.e-06]]
statePost: shape:(7, 1)
[[-9.2858   ]
 [-1.5959833]
 [ 2.874696 ]
 [ 0.1      ]
 [ 0.1      ]
 [ 0.1      ]
 [ 9.801    ]]
1111


In [5]:
kf.get_predict()

array([-9.2858   , -1.5959833,  2.874696 ], dtype=float32)

In [28]:
ball_pos_list =  [[-8.3324296128703, -1.426689529754115, 2.8019436403665923],[-7.459506668999277, -1.286501720742379, 2.720148095378055],[-6.540641694266555, -1.135716227096026, 2.6019241593327513],[-5.68329300302514, -0.9730271651650142, 2.519130845990981]]

In [7]:
kf.update(ball_pos[0], ball_pos[1], ball_pos[2], dT)

dT: 0.4000
---------------------------------------------------
meas current               : [-8.33243   -1.4266895  2.8019435]
pred predicted             : [-8.789115  -1.4913363  2.8583198]



In [8]:
ball_pos =  ball_pos_list.pop(0)
kf.update(ball_pos[0], ball_pos[1], ball_pos[2], dT)

dT: 0.4000
---------------------------------------------------
meas current               : [-7.4595065 -1.2865018  2.720148 ]
pred predicted             : [-7.4595075 -1.2865019  2.7201471]



In [9]:
ball_pos =  ball_pos_list.pop(0)
kf.update(ball_pos[0], ball_pos[1], ball_pos[2], dT)

dT: 0.4000
---------------------------------------------------
meas current               : [-6.540642  -1.1357162  2.6019242]
pred predicted             : [-6.5406413 -1.1357162  2.601923 ]



In [10]:
ball_pos =  ball_pos_list.pop(0)
kf.update(ball_pos[0], ball_pos[1], ball_pos[2], dT)

dT: 0.4000
---------------------------------------------------
meas current               : [-5.683293   -0.97302717  2.519131  ]
pred predicted             : [-5.683293   -0.97302717  2.5191298 ]



In [11]:
kf.predict(dT)

dT: 0.4000
---------------------------------------------------
meas current               : None (only predicting)
pred predicted without meas: [-4.8259444 -0.8103382  0.8681732]



In [12]:
kf.predict(dT)

dT: 0.4000
---------------------------------------------------
meas current               : None (only predicting)
pred predicted without meas: [-3.968596  -0.6476492 -2.3509436]



In [13]:
kf.get_predict()

array([-3.968596 , -0.6476492, -2.3509436], dtype=float32)

In [14]:
kf.KF.getPostState()

array([-3.968596 , -0.6476492, -2.3509436], dtype=float32)

In [31]:
np .array(ball_pos_list)[:,1][:-1]

array([-1.42668953, -1.28650172, -1.13571623, -0.97302717])

In [33]:
np.mean(np.array(ball_pos_list)[:,1][:-1])

-1.2829691591975065

In [34]:
a = [[[-9.039131345617234, -0.8137119203553631, 2.7444703070749017]], [[-7.900829857194978, -0.654493068471937, 2.6104207999239812]], [[-6.849655790366586, -0.5241522117656086, 2.4875451129360613]], [[-5.793390039377407, -0.39658750289812605, 2.3529254544909817]]]


In [46]:
b = np.array(a).reshape([-1,3])
b

array([[-9.03913135, -0.81371192,  2.74447031],
       [-7.90082986, -0.65449307,  2.6104208 ],
       [-6.84965579, -0.52415221,  2.48754511],
       [-5.79339004, -0.3965875 ,  2.35292545]])

In [50]:
sum(np.diff(b[:,1])) / 0.04

10.428110436430925

In [51]:
for i in []:
    print(1)

In [65]:
ball_cand_pos = [[-9.03913135, -0.81371192,  2.74447031],
       [-7.90082986, -0.65449307,  2.6104208 ],
       [-6.84965579, -0.52415221,  2.48754511],
       [-5.79339004, -0.3965875 ,  2.35292545]]

del_list = [1,2,3]

a= np.array(ball_cand_pos)

np.delete(a,del_list,axis = 0)

array([[-9.03913135, -0.81371192,  2.74447031]])

In [58]:
ball_cand_pos.pop(tuple(del_list))

TypeError: 'tuple' object cannot be interpreted as an integer

In [57]:
tuple(del_list)

(1, 2, 3)

In [66]:
if 3 == 3:
    print(1)

1


In [12]:
estimation_ball_trajectory_list =  np.array([[-8.075992233062022, -2.0712591029119727, 2.0143476038492176], [-7.041614424760605, -1.9654479963268496, 1.998508488845969], [-6.065044696824322, -1.879832764271466, 1.9635872373067076], [-5.107183755941063, -1.7968180783990304, 1.925614986159784], [-4.169832727705863, -1.6838107370497974, 1.884759164010521], [-3.2611958072772738, -1.5644043096331428, 1.81101188769961], [-2.368649316849956, -1.4805807124989778, 1.737028408823873], [-1.4834200661316506, -1.378965700223568, 1.6496367839315553], [-0.7006541039923906, -1.2862463955187806, 1.5580097756344515]])
estimation_ball_trajectory_list

array([[-8.07599223, -2.0712591 ,  2.0143476 ],
       [-7.04161442, -1.965448  ,  1.99850849],
       [-6.0650447 , -1.87983276,  1.96358724],
       [-5.10718376, -1.79681808,  1.92561499],
       [-4.16983273, -1.68381074,  1.88475916],
       [-3.26119581, -1.56440431,  1.81101189],
       [-2.36864932, -1.48058071,  1.73702841],
       [-1.48342007, -1.3789657 ,  1.64963678],
       [-0.7006541 , -1.2862464 ,  1.55800978]])

In [14]:
x_pos_list = estimation_ball_trajectory_list[:,0]
y_pos_list = estimation_ball_trajectory_list[:,1]
z_pos_list = estimation_ball_trajectory_list[:,2]


In [7]:
np.diff(estimation_ball_trajectory_list)

array([[ 6.00473313,  4.08560671],
       [ 5.07616643,  3.96395649],
       [ 4.18521193,  3.84342   ],
       [ 3.31036568,  3.72243306],
       [ 2.48602199,  3.5685699 ],
       [ 1.6967915 ,  3.3754162 ],
       [ 0.8880686 ,  3.21760912],
       [ 0.10445437,  3.02860248],
       [-0.58559229,  2.84425617]])

In [15]:
x_pos_list
np.diff(x_pos_list)[-1]

0.78276596213926

In [22]:
def cal_landing_point(pos_list):

    t_list = []

    if len(pos_list) < 4 : return [np.nan, np.nan, np.nan]

    pos = pos_list[-1]

    x0, y0, z0 = pos[0], pos[1], pos[2]

    vx, vy, vz = get_velocity(pos_list)

    a = -((0.5 * 0.507 * 1.2041 * np.pi * (0.033 ** 2) * vz ** 2 ) / 0.057 + 9.8 / 2 )
    b = vz
    c = z0

    t_list.append((-b + np.sqrt(b ** 2 - 4 * a * c))/(2 * a))
    t_list.append((-b - np.sqrt(b ** 2 - 4 * a * c))/(2 * a))

    t = max(t_list)
    
    x = np.array(x0 + vx * t - (0.5 * 0.507 * 1.2041 * np.pi * (0.033 ** 2) * vx ** 2 ) * (t ** 2) / 0.057,float)
    y = np.array(y0 + vy * t - (0.5 * 0.507 * 1.2041 * np.pi * (0.033 ** 2) * vy ** 2 ) * (t ** 2) / 0.057,float)
    z = np.array(z0 + vz * t - ((0.5 * 0.507 * 1.2041 * np.pi * (0.033 ** 2) * vz ** 2 ) / 0.057 + 9.8 / 2) * (t ** 2),float)
    
    return [np.round(x,3), np.round(y,3), np.round(z,3)]


def get_velocity(pos_list):
    
    t = 1 / 30
    
    np_pos_list = np.array(pos_list)

    x_pos_list = np_pos_list[:,0]
    y_pos_list = np_pos_list[:,1]
    z_pos_list = np_pos_list[:,2]  

    vel_x_list = np.diff(x_pos_list) / t
    vel_y_list = np.diff(y_pos_list) / t
    vel_z_list = np.diff(z_pos_list) / t

    return vel_x_list[-1], vel_y_list[-1], vel_z_list[-1]


In [23]:
ball_pos_jrajectory = [[-7.654350032583985, 0.37375046201544926, 1.5602039272816657], [-6.812516855329211, 0.4362023210314696, 1.6809758292885233], [-6.005632695613204, 0.5230876456730043, 1.7703496818844444], [-5.17434312120148, 0.5969135946437563, 1.842121410014573], [-4.40319049584631, 0.6610575247176333, 1.90162399321874], [-3.6256202508366666, 0.7485738124651196, 1.9551274411299535], [-2.8420151676001075, 0.8677493333923483, 1.980619130934988], [-2.058931199304543, 0.9710952377297057, 1.9892246369063118], [-1.3541772365570068, 1.0641107559204102, 1.9812870025634766], [-0.7198931574821472, 1.1478245258331299, 1.9584616422653198], [-0.14903749525547028, 1.223166823387146, 1.9222372770309448]]
ball_pos_jrajectory

[[-7.654350032583985, 0.37375046201544926, 1.5602039272816657],
 [-6.812516855329211, 0.4362023210314696, 1.6809758292885233],
 [-6.005632695613204, 0.5230876456730043, 1.7703496818844444],
 [-5.17434312120148, 0.5969135946437563, 1.842121410014573],
 [-4.40319049584631, 0.6610575247176333, 1.90162399321874],
 [-3.6256202508366666, 0.7485738124651196, 1.9551274411299535],
 [-2.8420151676001075, 0.8677493333923483, 1.980619130934988],
 [-2.058931199304543, 0.9710952377297057, 1.9892246369063118],
 [-1.3541772365570068, 1.0641107559204102, 1.9812870025634766],
 [-0.7198931574821472, 1.1478245258331299, 1.9584616422653198],
 [-0.14903749525547028, 1.223166823387146, 1.9222372770309448]]

In [24]:
cal_landing_point(ball_pos_jrajectory)

[7.352, 2.382, 0.0]

In [26]:
a = np.array([    -3.1019,     -2.3294,      -1.513])

In [29]:
np.mean(np.diff(a))/(1/25)

19.861250000000002

In [28]:
np.diff(a)/(1/25)

array([0.7725, 0.8164])

In [46]:
x0, y0, z0 =  -0.14778512716293335, -3.731870412826538, 2.6436607837677
vx, vy, vz =  13.437911868095398, 0.15355348587036133, 0.08598566055297852



In [49]:
t_list = []
a = -((0.5 * 0.507 * 1.2041 * np.pi * (0.033 ** 2) * vz ** 2 ) / 0.057 + 9.8 / 2 )
b = vz
c = z0

t_list.append((-b + np.sqrt(b ** 2 - 4 * a * c))/(2 * a))
t_list.append((-b - np.sqrt(b ** 2 - 4 * a * c))/(2 * a))

t = max(t_list)

drag_x = (0.5 * 0.507 * 1.2041 * np.pi * (0.033 ** 2) * vx ** 2 )
drag_y = (0.5 * 0.507 * 1.2041 * np.pi * (0.033 ** 2) * vy ** 2 )
drag_z = (0.5 * 0.507 * 1.2041 * np.pi * (0.033 ** 2) * vz ** 2 )

drag_x = 0
drag_y = 0
drag_z = 0

x = np.array(x0 + vx * t - drag_x * (t ** 2) / 0.057,float)
y = np.array(y0 + vy * t - drag_y * (t ** 2) / 0.057,float)
z = np.array(z0 + vz * t - (drag_z / 0.057 + 9.8 / 2) * (t ** 2),float)

[np.round(x,3), np.round(y,3), np.round(z,3)]


[9.841, -3.618, 0.0]

In [35]:
t_list

[-0.7671826894274586, 0.7029915699965572]